# Step 2 - Launch the Training Job

In this notebook, we will use the cluster created in **[Step 0 - Set up the Cluster](SetupCluster.ipynb)** to train the reinforcement learning model. 

## The experiment architecture

Although reinforcment learning is powerful, the algorithms take a long time to train. To speed up the process, we provision multiple machines in the cluster. We assign one of the machines to be the parameter server node, and the rest of the machines to be agent nodes. The parameter server is responsible for keeping track of the master copy of the model. The agent nodes each receive a copy of the model from the parameter server and perform a training iteration locally. Once its individual training iteration has completed, an agent sends its weight updates (the "gradients") to the parameter server. The parameter server then replicates the gradient update, and sends out the newly updated model to the agent node for the next iteration. The updates happen asynchronously between nodes. Periodically, the parameter server will save the model to the file share. Below is a graphical representation of the experiment architecture.

![experiment_architecture](experiment_architecture.png)

Let's start by importing some libraries to launch the training job.

In [3]:
import os
import sys
import uuid
import json

#Azure batch. To install, run 'pip install cryptography azure-batch azure-storage'
import azure.batch.batch_service_client as batch
import azure.batch.batch_auth as batchauth
import azure.batch.models as batchmodels

with open('notebook_config.json', 'r') as f:
    NOTEBOOK_CONFIG = json.loads(f.read()) 

Now, we will define some hyperparameters for the training job. The parameters are:

* **batch_update_frequency**: This is how often the weights from the actively trained network get copied to the target network. It is also how often the model gets saved to disk. For more details on how this works, check out the [Deep Q-learning paper](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf).
* **max_epoch_runtime_sec**: This is the maximum runtime for each epoch. If the car has not reached a terminal state after this many seconds, the epoch will be terminated and training will begin.
* **per_iter_epsilon_reduction**: The agent uses an epsilon greedy linear annealing strategy while training. This is the amount by which epsilon is reduced each iteration.
* **min_epsilon**: The minimum value for epsilon. Once reached, the epsilon value will not decrease any further.
* **batch_size**: The minibatch size to use for training.
* **replay_memory_size**: The number of examples to keep in the replay memory. The replay memory is a FIFO buffer used to reduce the effects of nearby states being correlated. Minibatches are generated from randomly selecting examples from the replay memory.
* **weights_path**: If we are doing transfer learning and using pretrained weights for the model, they will be loaded from this path.
* **train_conv_layers**: If we are using pretrained weights, we may prefer to freeze the convolutional layers to speed up training.

In [4]:
batch_update_frequency = 300
max_epoch_runtime_sec = 30
per_iter_epsilon_reduction=0.003
min_epsilon = 0.1
batch_size = 32
replay_memory_size = 2000
weights_path = 'Z:\\data\\pretrain_model_weights.h5'
train_conv_layers = 'false'

Connect to the Azure Batch service and create a unique job name

In [5]:
batch_credentials = batchauth.SharedKeyCredentials(NOTEBOOK_CONFIG['batch_account_name'], NOTEBOOK_CONFIG['batch_account_key'])
batch_client = batch.BatchServiceClient(batch_credentials, base_url=NOTEBOOK_CONFIG['batch_account_url'])

job_id = 'distributed_rl_{0}'.format(str(uuid.uuid4()))

Next, we create the job. 

In [6]:
job = batch.models.JobAddParameter(
        job_id,
        batch.models.PoolInformation(pool_id=NOTEBOOK_CONFIG['batch_pool_name']))

batch_client.job.add(job)

Although we've created the job, we haven't actually told the machines what to do. For that, we need to create tasks in the job. Each machine will pick up a different task. We create one task for the parameter server node, and one task for each of the agent nodes.

In [7]:
tasks = []

# Trainer task
tasks.append(batchmodels.TaskAddParameter(
        id='TrainerTask',
        command_line=r'call C:\\prereq\\mount.bat && C:\\ProgramData\\Anaconda3\\Scripts\\activate.bat py36 && python -u Z:\\scripts_downpour\\manage.py runserver 0.0.0.0:80 data_dir=Z:\\\\ role=trainer experiment_name={0} batch_update_frequency={1} weights_path={2} train_conv_layers={3} per_iter_epsilon_reduction={4} min_epsilon={5}'.format(job_id, batch_update_frequency, weights_path, train_conv_layers, per_iter_epsilon_reduction, min_epsilon),
        display_name='Trainer',
        user_identity=batchmodels.UserIdentity(user_name=NOTEBOOK_CONFIG['batch_job_user_name']),
        multi_instance_settings = batchmodels.MultiInstanceSettings(number_of_instances=1, coordination_command_line='cls')
    ))

# Agent tasks
agent_cmd_line = r'call C:\\prereq\\mount.bat && C:\\ProgramData\\Anaconda3\\Scripts\\activate.bat py36 && python -u Z:\\scripts_downpour\\app\\distributed_agent.py data_dir=Z: role=agent max_epoch_runtime_sec={0} per_iter_epsilon_reduction={1:f} min_epsilon={2:f} batch_size={3} replay_memory_size={4} experiment_name={5} weights_path={6} train_conv_layers={7}'.format(max_epoch_runtime_sec, per_iter_epsilon_reduction, min_epsilon, batch_size, replay_memory_size, job_id, weights_path, train_conv_layers) 
for i in range(0, NOTEBOOK_CONFIG['batch_pool_size'] - 1, 1):
    tasks.append(batchmodels.TaskAddParameter(
            id='AgentTask_{0}'.format(i),
            command_line = agent_cmd_line,
            display_name='Agent_{0}'.format(i),
            user_identity=batchmodels.UserIdentity(user_name=NOTEBOOK_CONFIG['batch_job_user_name']),
            multi_instance_settings=batchmodels.MultiInstanceSettings(number_of_instances=1, coordination_command_line='cls')
        ))
    
batch_client.task.add_collection(job_id, tasks)
print('')

Now the job has been kicked off! Shortly, you should see two new directories created on the file share:

* **logs**: This contains the stdout for the agent and the trainer nodes. These streams are very useful for debugging. To add additional debug information, just print() to either stdout or stderr in the training code. 
* **checkpoint**: This contains the trained models. After the required number of minibatches have been trained (as determined by the batch_update_frequency parameter), the model's weights will be saved to this directory on disk. 

In each of these folders, a subdirectory will be created with your experiment Id. 

If you use remote desktop to connect to the agent machines, you will be able to see the training code drive the vehicle around (be sure to give administrator privileges to run any powershell scripts when prompted). 

Training will continue indefinitely. Be sure to let the model train for at least 300,000 iterations. Once the model has trained, download the weights and move on to **[Step 3 - Run the Model](RunModel.ipynb)**.